In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

import nfn_crawring as nf
import gbq_service as gbq

CORP = ['054950', '제이브이엠']

In [2]:
# 한국전자금융 연간 손익계산서
df = nf.crawl_nfn(stock_code=CORP[0],
             type=nf.DataType.FS,
             subType=nf.FSType.IS,
             period=nf.Period.QUARTER)
df

https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=054950&frq=1&rpt=0&finGubun=MAIN&frqTyp=1&cn=&encparam=QkhpMzRiUTA2ZzVoK3pBUmpiL0laQT09


,account_nm,2022/09,2022/12,2023/03,2023/06,2023/09
0,매출액(수익),341.671661,392.855313,376.573656,387.147635,368.466413
1,*내수,179.365250,213.488180,193.907500,207.573980,204.627140
2,*수출,162.306420,179.367130,182.666160,179.573660,163.839260
3,....제품매출액,NaN,NaN,NaN,349.375634,NaN
4,....상품매출액,NaN,NaN,NaN,37.772001,NaN
...,...,...,...,...,...,...
239,*희석주당순이익,NaN,NaN,NaN,NaN,NaN
240,*(지배주주지분)주당계속사업이익,399.000000,NaN,NaN,290.000000,466.000000
241,*(지배주주지분)주당순이익,399.000000,NaN,NaN,290.000000,466.000000
242,*(지배주주지분)희석주당계속사업이익,NaN,NaN,NaN,NaN,NaN


In [7]:
def _convert(crawring_df,
             stock_code,
             corp_name,
             type):

    df = crawring_df.copy()

    # 순서 저장 후 짧은 구조 변환
    df['order'] = df.index.to_list()
    df = df.melt(['order', 'account_nm'])

    # 추가 데이터 세팅
    df['stock_code'] = stock_code
    df['corp_name'] = corp_name
    df['type'] = type
    df['updated'] = datetime.now()

    # 연, 분기 데이터 세팅
    df.columns = ['order', 'account_name', 'date', 'amount', 'stock_code', 'corp_name', 'type', 'updated']
    df['year'] = df.date.str.split('/').str[0].astype(int)
    df['quarter'] = np.ceil(df.date.str.split('/').str[1].astype(int) / 3).astype(int)
    
    # 컬럼 정리
    df = df[['stock_code', 'corp_name', 'year', 'quarter', 'type', 'order', 'account_name', 'amount', 'updated']]

    # year 변환 및 id 생성
    df.index = df.stock_code + '_' + df.year.astype(str) + '_' + df.quarter.astype(str) + '_' + df.type + '_' + df.order.astype(str).str.zfill(3)
    df.index.name = 'id'

    return df

In [8]:
_convert(df, CORP[0], CORP[1], 'IS')

,stock_code,corp_name,year,quarter,type,order,account_name,amount,updated
id,,,,,,,,,
054950_2022_3_IS_000,054950,제이브이엠,2022,3,IS,0,매출액(수익),341.671661,2024-01-25 17:23:35.158692
054950_2022_3_IS_001,054950,제이브이엠,2022,3,IS,1,*내수,179.365250,2024-01-25 17:23:35.158692
054950_2022_3_IS_002,054950,제이브이엠,2022,3,IS,2,*수출,162.306420,2024-01-25 17:23:35.158692
054950_2022_3_IS_003,054950,제이브이엠,2022,3,IS,3,....제품매출액,NaN,2024-01-25 17:23:35.158692
054950_2022_3_IS_004,054950,제이브이엠,2022,3,IS,4,....상품매출액,NaN,2024-01-25 17:23:35.158692
...,...,...,...,...,...,...,...,...,...
054950_2023_3_IS_239,054950,제이브이엠,2023,3,IS,239,*희석주당순이익,NaN,2024-01-25 17:23:35.158692
054950_2023_3_IS_240,054950,제이브이엠,2023,3,IS,240,*(지배주주지분)주당계속사업이익,466.000000,2024-01-25 17:23:35.158692
054950_2023_3_IS_241,054950,제이브이엠,2023,3,IS,241,*(지배주주지분)주당순이익,466.000000,2024-01-25 17:23:35.158692
